In [ ]:
# !pip install boto3

<h2>Single Turn invocations</h2>

In [ ]:
import boto3

client_runtime = boto3.client('bedrock-agent-runtime')

user_input = """What's a good place nearby for brunch with a nice view?"""

response = client_runtime.invoke_flow(
    flowIdentifier="",
    flowAliasIdentifier="",
    inputs=[
        {
            "content": {"document":str(user_input)},
            "nodeName": "FlowInputNode",
            "nodeOutputName": "document"
        }
    ]
)
result = {}

for event in response.get("responseStream"):
    result.update(event)

if result['flowCompletionEvent']['completionReason'] == 'SUCCESS':
    print("Flow invocation was successful! The output of the flow is as follows:\n")
    print(result['flowOutputEvent']['content']['document'])

else:
    print("The flow invocation completed because of the following reason:", result['flowCompletionEvent']['completionReason'])

Flow invocation was successful! The output of the flow is as follows:

The Blue Garden Café, located 850m from the hotel, would be a good option for brunch. They serve fresh juices, artisanal pastries, and offer vegan-friendly options. It's described as perfect for breakfast and casual meetups. If you're willing to go a bit further, you could visit Palm Bay Waterfront Park (3.4km away) after your meal for scenic views and walking trails. However, the search results don't specifically mention any restaurants with views, so I cannot make a specific recommendation for a brunch spot with a view.


<h2>Multi-turn conversations</h2>

In [2]:
import boto3
import logging
import json

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

FLOW_ID = "JLGXMMQLD7"
FLOW_ALIAS_ID = "0VRWTC4AIX"

def invoke_flow(client, flow_id, flow_alias_id, input_data, execution_id=None):
    """
    Invoke an Amazon Bedrock flow and gather:
      - execution_id (for session continuity)
      - flow_status (SUCCESS, INPUT_REQUIRED, etc.)
      - any multi-turn request event
      - any final output document
    """
    params = {
        "flowIdentifier": flow_id,
        "flowAliasIdentifier": flow_alias_id,
        "inputs": [input_data],
    }
    if execution_id:
        params["executionId"] = execution_id

    try:
        resp = client.invoke_flow(**params)
        
        execution_id = resp.get("executionId", execution_id)
        if not execution_id:
            logger.error("InvokeFlow did not return an executionId. Check flow configuration or permissions.")
            return None, None, None, None 

        flow_status = None
        multi_turn_request = None
        final_output = None

        for evt in resp.get("responseStream", []):
            logger.debug(f"Received event: {json.dumps(evt, indent=2)}")

            if "flowCompletionEvent" in evt:
                flow_status = evt["flowCompletionEvent"]["completionReason"]
                logger.info(f"Flow Completion Event received with reason: {flow_status}")

            if "flowMultiTurnInputRequestEvent" in evt:
                multi_turn_request = evt["flowMultiTurnInputRequestEvent"]
                logger.info("Received flowMultiTurnInputRequestEvent - input required.")

            if "flowOutputEvent" in evt:
                content = evt["flowOutputEvent"].get("content", {})
                document_content = content.get("document")
                if isinstance(document_content, dict) and 'content' in document_content:
                    final_output = document_content.get('content')
                elif isinstance(document_content, str):
                    final_output = document_content 
                else:
                    final_output = json.dumps(document_content)
                logger.info("Received flowOutputEvent - final output available.")

        return flow_status, multi_turn_request, final_output, execution_id

    except botocore.exceptions.ClientError as e:
        logger.error(f"AWS Client Error invoking flow: {e}")
        logger.error(f"Error Code: {e.response.get('Error', {}).get('Code')}, Message: {e.response.get('Error', {}).get('Message')}")
        return None, None, None, None 
    except Exception as e:
        logger.error(f"Unexpected error in invoke_flow: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None, None, None, None 

def create_input_data(text, node_name="FlowInputNode", is_initial=True):
    """
    Build the inputs entry for invoke_flow.
    Handles initial FlowInputNode (expects string document) and subsequent node inputs.
    """
    data = {
        "nodeName": node_name,
    }
    if is_initial:

        data["content"] = {"document": str(text)}
        data["nodeOutputName"] = "document"
    else:
        if node_name == "hotel_booking_agent": # Target node is the hotel booking agent
            data["content"] = {"document": str(text)}
            data["nodeInputName"] = "agentInputText"
        else:
            data["content"] = {"document": {"mimeType": "text/plain", "content": str(text)}}
            data["nodeInputName"] = "agentInputText" 
    return data

def main():
    client = boto3.client("bedrock-agent-runtime")
    current_execution_id = None 
    node_for_next_turn = "FlowInputNode" # Default for the very first turn

    print("\n--- Start Conversation ---")
    user_input = input("You: ")
    
    while True:
        is_initial_turn = (current_execution_id is None)

        if is_initial_turn:
            input_data = create_input_data(user_input, node_name="FlowInputNode", is_initial=True)
        else:
            input_data = create_input_data(user_input, node_name=node_for_next_turn, is_initial=False)

        status, multi_request, final_output, current_execution_id = invoke_flow(
            client, FLOW_ID, FLOW_ALIAS_ID, input_data, current_execution_id
        )

        if current_execution_id is None:
            print("\n Failed to start/continue flow. Exiting.")
            break

        if status == "SUCCESS":
            print("\n Flow completed successfully:\n")
            if final_output:
                print(final_output)
            else:
                print("Flow completed successfully but no final output document was found.")
            break 

        elif status == "INPUT_REQUIRED" and multi_request:
            prompt = multi_request["content"]["document"]
            node_for_next_turn = multi_request["nodeName"]
            
            print(f"\nAgent: '{prompt}'")
            user_input = input("You: ") 
            continue 

        else:
            logger.error(f"Flow ended with unexpected reason: {status}. Output: {final_output}")
            break

if __name__ == "__main__":
    main()


--- Start Conversation ---


You:  i want to know if i have any existing bookings?


INFO:__main__:Received flowMultiTurnInputRequestEvent - input required.
INFO:__main__:Flow Completion Event received with reason: INPUT_REQUIRED



Agent: 'To check your existing bookings, I'll need either your email address or phone number. Could you please provide either or both?'


You:  carol.lee@example.com


INFO:__main__:Received flowOutputEvent - final output available.
INFO:__main__:Flow Completion Event received with reason: SUCCESS



 Flow completed successfully:

Yes, I found one confirmed booking under your email:
- Check-in: April 28, 2025
- Check-out: May 3, 2025
- Number of guests: 3
- Booking ID: bkg-1003
